In [ ]:
import numpy as np
import pandas as pd

from pyspark.ml.image import ImageSchema

from pyspark.ml.classification import MultilayerPerceptronClassifier,MultilayerPerceptronClassificationModel
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [ ]:
img_dir = 'dbfs:/mnt/<任意のディレクトリ名>'

image_df = ImageSchema.readImages(img_dir)

display(image_df) 

image_row = image_df.collect()[0]
image_array = ImageSchema.toNDArray(image_row.image)

In [ ]:
image_array = np.rot90(image_array, k=1)
image_array = image_array[::-1]

ary =[]
for k in range(280):
  bbb = []
  for i in range(28):
    aaa = 0
    for j in range(10):
      aaa += image_array[k][10*i+j][0]
    bbb.append(int(aaa/10))
  ary.append(bbb)

ary2 = []
for k in range(28):
  ccc = []
  for i in range(28):
    aaa = 0
    for j in range(10):
      aaa += ary[10*k+j][i]
    ccc.append(int(aaa/10))
  ary2.append(ccc)

im = []
for i in range(28):
  m = []
  for j in range(28):
    m = 255-ary2[i][j]
    im.append(m)
im = [im]

In [ ]:
def open_csv(path):

  d = spark.read.format("csv")\
    .option("header","false")\
    .option("inferSchema","true")\
    .load(path)

  return d

test_csv = open_csv("dbfs:/FileStore/tables/emnist_balanced_test-dc5d0.csv")

col = test_csv.columns[1:]

pdf = pd.DataFrame(im, columns=col)
spark.conf.set("spark.sql.execution.arrow.enabled", "true")
df = spark.createDataFrame(pdf)

assembler = VectorAssembler(inputCols=df.columns,outputCol="features")
data = assembler.transform(df)

In [ ]:
mp_model = MultilayerPerceptronClassificationModel.load("/MPmodel")
result = mp_model.transform(data)

In [ ]:
ans =["0","1","2","3","4","5","6","7","8","9","A","B","C","D","E","F","G","H","I","J","K","L","M","N","O","P","Q","R","S","T","U","V","W","X","Y","Z","a","b","d","e","f","g","h","n","q","r","t"]
pre = result.select("prediction").rdd.flatMap(lambda x: x).collect()
pre = int(pre[0])
print(ans[pre])

In [ ]:
dbutils.notebook.exit(ans[pre])